WORKFLOW:

- for every row in statcast 
- grab game_pk 
- dump into statsapi.schedule 
- grab starttime, venue 
- translate venue into lat, long with coord.csv 
- dump into visual crossing 
- grab weather info 
- dump starttime, venue, and weather info back into statcast df 
- grab pitcher id from df 
- run through pybaseball.statcast_pitcher to grab "relevent" stats through DAY BEFORE GAME 
- dump stats in df 
- grab batter id from df 
- run through pybaseball.statcast_batter to grab "relevent" stata through DAY BEFORE GAME 
- dump stats in df

IMPLEMENT CACHING - have relevent parts of code check their caches first:

save game_pk results and their matching starttime, location, weather - no need to ever clear

save pitcher and batter stats - clear after every day (not game because you can't wager between double headers)


according to retrosheets the day/night cutoff is 5PM

Import data from statcast

In [2]:
from pybaseball import statcast, statcast_pitcher, statcast_batter, playerid_lookup
import pandas as pd
pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', 10)
import numpy as np
import statsapi
from datetime import datetime, timedelta
from dateutil import tz
from geopy.geocoders import Nominatim


In [ ]:
data = statcast('2019-03-20', '2019-09-29')

data.to_csv('2019_statcast_pbp.csv', index=False)

data = statcast('2020-07-23', '2020-09-27')

data.to_csv('2020_statcast_pbp.csv', index=False)

data = statcast('2021-04-01', '2021-06-30')

data.to_csv('2021_statcast_pbp.csv', index=False)

In [2]:
df = pd.concat([pd.read_csv('2019_statcast_pbp.csv'),
                pd.read_csv('2020_statcast_pbp.csv'),
               pd.read_csv('2021_statcast_pbp.csv')], ignore_index=True)

In [ ]:
statcast('2018-01-01', '2018-12-31').to_csv('2018_statcast_pbp.csv', index=False)
statcast('2017-01-01', '2017-12-31').to_csv('2017_statcast_pbp.csv', index=False)


In [3]:
df['bottom'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0)

In [4]:
df = df.drop(labels=['inning_topbot'], axis=1)

In [ ]:
# df['game_date'] = pd.to_datetime(df['game_date'])
# df['game_date'] = df['game_date'].apply(lambda x: x.toordinal())

In [5]:
df = df.sort_values(['game_date', 'game_pk','home_team', 'inning', 'bottom', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)

In [ ]:
df.head(20)

In [ ]:
df.game_type.value_counts()

In [ ]:
df.description.value_counts()

In [ ]:
df.events.value_counts()

In [ ]:
df.head()

In [ ]:
from pybaseball import playerid_reverse_lookup

playerid_reverse_lookup([547982, 543208], key_type='mlbam')

In [ ]:
df[df['events'] == 'other_out']

In [ ]:
# class PlateAppearence:
#     def __init__(self, row):
#         self.balls = row['balls']
#         self.strikes = row['strikes']
#         self.batter = row['batter']
#         self.pitcher = row['pitcher']
#         self.inning = row['inning']
#         self.bottom = row['bottom']
#         self.outs = row['outs_when_up']
        
    

In [ ]:
# for index, row in df.iterrows():
#     pa = PlateAppearence(row)
    
#     try:
#         pa == 
#     prev_row = row

In [8]:
pa_ending_events = np.array(['field_out',
                    'strikeout',
                    'single',
                    'walk',
                    'double',
                    'home_run',
                    'force_out',
                    'grounded_into_double_play',
                    'hit_by_pitch',
                    'field_error',
                    'sac_fly',
                    'triple',
                    'sac_bunt',
                    'fielders_choice',
                    'double_play',
                    'fielders_choice_out',
                    'strikeout_double_play',
                    'catcher_interf',
                    'sac_fly_double_play',
                    'triple_play',
                    'sac_bunt_double_play'])

df_filtered = df[df['events'].isin(pa_ending_events)]

NameError: name 'df' is not defined

In [9]:
hit_events = np.array(['single',
            'double',
            'home_run',
            'triple'])

In [ ]:
df_filtered

Sanity test to make sure rows are in expected order

In [ ]:
previous_batter = {'batter': 0}

for index, row in df_filtered.iterrows():
    if previous_batter['batter'] != row['batter']:
        previous_batter = row
        previous_index = index
        pass
    elif previous_batter['inning'] == 9 and previous_batter['outs_when_up'] == 2 and row['at_bat_number'] == 1:
            previous_batter = row
            previous_index = index
            pass
    else:
        print(previous_index, previous_batter)
        print(index, row)
        break

load in Parks.csv as pd

In [8]:
venue_coords = pd.read_csv('Parks.csv')

In [ ]:
venue_coords

In [9]:
game_pk_df = pd.DataFrame(columns = statsapi.schedule(game_id=566083)[0].keys())

In [10]:
venue_dict = {}

for index, row in df_filtered.iterrows():
    game_pk = row['game_pk']
    if game_pk in game_pk_df['game_id'].unique():
        starttime = game_pk_df[game_pk_df['game_id'] == game_pk]['game_datetime'].iloc[0]
        venue = game_pk_df[game_pk_df['game_id'] == game_pk]['venue_name'].iloc[0]
    else:
        game_data = statsapi.schedule(game_id = game_pk)[0]
        starttime = game_data['game_datetime']
        venue = game_data['venue_name']
        game_pk_df = game_pk_df.append(game_data, ignore_index=True)
        print('***************', venue)
#     if venue in ['Tokyo Dome',
#                  'Estadio de Beisbol Monterrey',
#                  'TD Ameritrade Park',
#                  'London Stadium',
#                  'BB&T Ballpark']:
#         pass
#     else:
    try:
        city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
        venue_dict[venue] = np.array((city, lat, lon, alt))
    except IndexError:
        raise IndexError(venue, index)
#     print(venue_dict)

*************** Tokyo Dome
*************** Tokyo Dome
*************** Great American Ball Park
*************** Kauffman Stadium
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
*************** Globe Life Park in Arlington
*************** Rogers Centre
*************** Target Field
*************** Citizens Bank Park
*************** Marlins Park
*************** Yankee Stadium
*************** Miller Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
*************** Rogers Centre
*************** Marlins Park
*************** Miller Park
*************** Kauffman Stadium
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** Petco Park
*************** T-Mobile Park
*************** Tropicana Field
****

*************** Nationals Park
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Globe Life Park in Arlington
*************** Target Field
*************** Citizens Bank Park
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
*************** Yankee Stadium
*************** Miller Park
*************** Angel Stadium
*************** Coors Field
*************** Comerica Park
*************** Nationals Park
*************** Petco Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Yankee Stadium
*************** Miller Park
*************** Angel Stadium
*************** Wrigley Field
*************** Coors Field
*************** Comerica Park
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Globe Life Park in Arlington
***

*************** Oriole Park at Camden Yards
*************** Wrigley Field
*************** Progressive Field
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Rogers Centre
*************** Yankee Stadium
*************** Miller Park
*************** Oriole Park at Camden Yards
*************** Wrigley Field
*************** Progressive Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** PNC Park
*************** Petco Park
*************** Busch Stadium
*************** Tropicana Field
*************** Rogers Centre
*************** Yankee Stadium
*************** Miller Park
*************** Chase Field
*************** Wrigley Field
*************** Progressive Field
****

*************** PNC Park
*************** Oracle Park
*************** Busch Stadium
*************** Rogers Centre
*************** Target Field
*************** Miller Park
*************** Oriole Park at Camden Yards
*************** Fenway Park
*************** Great American Ball Park
*************** Great American Ball Park
*************** Coors Field
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Target Field
*************** Guaranteed Rate Field
*************** Yankee Stadium
*************** Oriole Park at Camden Yards
*************** Fenway Park
*************** Great American Ball Park
*************** Coors Field
*************** Minute Maid Park
*************** Dodger Stadium
*************** Oakland Coliseum
*************** T-Mobile Park
*************** Tropicana Field
*************** Target Field
*************** Citizens Bank P

*************** Marlins Park
*************** Oriole Park at Camden Yards
*************** Great American Ball Park
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Citi Field
*************** Oakland Coliseum
*************** Oracle Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
*************** Oriole Park at Camden Yards
*************** Great American Ball Park
*************** Coors Field
*************** Comerica Park
*************** Minute Maid Park
*************** Dodger Stadium
*************** Nationals Park
*************** Citi Field
*************** Oakland Coliseum
*************** Oracle Park
*************** Tropicana Field
*************** Target Field
*************** SunTrust Park
*************** Guaranteed Rate Field
*************** Marlins Park
***

...

*************** Wrigley Field
*************** Angel Stadium
*************** Fenway Park
*************** loanDepot park
*************** Yankee Stadium
*************** Comerica Park
*************** Citizens Bank Park
*************** T-Mobile Park
*************** Coors Field
*************** Nationals Park
*************** Oakland Coliseum
*************** Globe Life Field
*************** Nationals Park
*************** Yankee Stadium
*************** Coors Field
*************** Wrigley Field
*************** Petco Park
*************** Oakland Coliseum
*************** Great American Ball Park
*************** Globe Life Field
*************** loanDepot park
*************** Progressive Field
*************** T-Mobile Park
*************** Nationals Park
*************** Comerica Park
*************** Citizens Bank Park
*************** Fenway Park
*************** Oriole Park at Camden Yards
*************** TD Ballpark
*************** Minute Maid Park
*************** Target Field
*************** PNC Par

*************** Minute Maid Park
*************** Progressive Field
*************** Truist Park
*************** Oriole Park at Camden Yards
*************** Comerica Park
*************** American Family Field
*************** Busch Stadium
*************** American Family Field
*************** PNC Park
*************** Guaranteed Rate Field
*************** Minute Maid Park
*************** TD Ballpark
*************** Progressive Field
*************** Truist Park
*************** Oracle Park
*************** Globe Life Field
*************** Chase Field
*************** Tropicana Field
*************** Dodger Stadium
*************** Oriole Park at Camden Yards
*************** Citi Field
*************** Tropicana Field
*************** Oracle Park
*************** Citi Field
*************** Busch Stadium
*************** PNC Park
*************** American Family Field
*************** Chase Field
*************** TD Ballpark
*************** Globe Life Field
*************** Minute Maid Park
**************

*************** Minute Maid Park
*************** Minute Maid Park
*************** TD Ballpark
*************** Tropicana Field
*************** Comerica Park
*************** Oriole Park at Camden Yards
*************** T-Mobile Park
*************** Petco Park
*************** PNC Park
*************** Coors Field
*************** Fenway Park
*************** Dodger Stadium
*************** Guaranteed Rate Field
*************** Chase Field
*************** American Family Field
*************** Target Field
*************** Petco Park
*************** Truist Park
*************** Great American Ball Park
*************** Wrigley Field
*************** Target Field
*************** T-Mobile Park
*************** Dodger Stadium
*************** Globe Life Field
*************** Angel Stadium
*************** Kauffman Stadium
*************** Oakland Coliseum
*************** Citizens Bank Park
*************** Wrigley Field
*************** Great American Ball Park
*************** Petco Park
*************** Glob

*************** Globe Life Field
*************** Angel Stadium
*************** Kauffman Stadium
*************** Guaranteed Rate Field
*************** American Family Field
*************** PNC Park
*************** Citizens Bank Park
*************** Coors Field
*************** PNC Park
*************** Citizens Bank Park
*************** Busch Stadium
*************** Kauffman Stadium
*************** Sahlen Field
*************** Oriole Park at Camden Yards
*************** Petco Park
*************** American Family Field
*************** Truist Park
*************** Globe Life Field
*************** Angel Stadium
*************** Coors Field
*************** Yankee Stadium
*************** Oracle Park
*************** Guaranteed Rate Field
*************** Guaranteed Rate Field
*************** Sahlen Field
*************** Citizens Bank Park
*************** PNC Park
*************** American Family Field
*************** Globe Life Field
*************** Petco Park
*************** Busch Stadium
********

*************** Dodger Stadium
*************** Great American Ball Park
*************** Tropicana Field
*************** Target Field
*************** loanDepot park
*************** Yankee Stadium
*************** Sahlen Field
*************** Comerica Park
*************** Petco Park
*************** Busch Stadium
*************** Great American Ball Park
*************** Oracle Park
*************** Tropicana Field
*************** Sahlen Field
*************** Fenway Park
*************** Citi Field
*************** Target Field
*************** Globe Life Field
*************** loanDepot park
*************** American Family Field
*************** Dodger Stadium
*************** Guaranteed Rate Field
*************** Citi Field
*************** Oracle Park
*************** Guaranteed Rate Field
*************** Comerica Park
*************** Great American Ball Park
*************** Petco Park
*************** American Family Field
*************** Sahlen Field
*************** Tropicana Field
**************

In [ ]:
df_filtered.tail()

In [ ]:
pd.DataFrame(venue_dict).to_csv('venue_dict')
df_filtered.to_csv('df_filtered')
game_pk_df.to_csv('game_pk_df')

In [ ]:
venue_dict = pd.read_csv('venue_dict').drop('Unnamed: 0', axis = 1).to_dict()
df_filtered = pd.read_csv('df_filtered')
game_pk_df = pd.read_csv('game_pk_df')

weather work

In [ ]:
game_pk_df

In [11]:
game_pk_df = game_pk_df.sort_values(['game_datetime'])

In [ ]:
len(game_pk_df)

In [ ]:
len(datetime_coordinate_matching.keys())/999

In [ ]:
venue_dict

In [12]:
from datetime import datetime, timedelta
from dateutil import tz
from geopy.geocoders import Nominatim

def convert_UTC_to_local(row):
    venue_name = row['venue_name']
    game_datetime = row['game_datetime']
    from_zone = tz.gettz('UTC')
    
    locator = Nominatim(user_agent='myGeocoder')
    city, lat, lon, alt = venue_dict[venue_name]
#     coordinates = ', '.join([str(lat), str(lon)])
#     location = locator.reverse(coordinates)
#     print(location.raw)
#     venue_city = location.raw['address']['city']
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                        'Oakland',
                        'Phoenix',
                        'Seattle',
                        'Los Angeles',
                        'San Diego',
                        'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                        'Milwaukee',
                        'Chicago',
                        'St. Louis',
                        'Arlington',
                        'Kansas City',
                        'Houston',
                        'Monterrey',
                        'Omaha']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                        'Detroit',
                        'Cincinnati',
                        'Pittsburgh',
                        'Tampa Bay',
                        'Philadelphia',
                        'Atlanta',
                        'New York',
                        'Washington',
                        'Cleveland',
                        'Miami',
                        'Boston',
                        'Baltimore',
                        'Toronto',
                        'Williamsport',
                        'Dunedin',
                        'St. Petersburg']:
        to_zone = tz.gettz('America/New_York')
    else:
        raise IndexError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    local = utc.astimezone(to_zone)
    print(local)
    return(local)

game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)

2019-03-20 18:35:00+09:00
2019-03-21 18:35:00+09:00
2019-03-28 13:05:00-04:00
2019-03-28 13:05:00-04:00
2019-03-28 13:10:00-05:00
2019-03-28 15:05:00-04:00
2019-03-28 15:37:00-04:00
2019-03-28 16:00:00-04:00
2019-03-28 15:05:00-05:00
2019-03-28 13:07:00-07:00
2019-03-28 13:10:00-07:00
2019-03-28 15:10:00-05:00
2019-03-28 13:10:00-07:00
2019-03-28 16:10:00-04:00
2019-03-28 16:10:00-04:00
2019-03-28 15:15:00-05:00
2019-03-28 16:10:00-07:00
2019-03-29 19:07:00-04:00
2019-03-29 19:10:00-04:00
2019-03-29 19:10:00-04:00
2019-03-29 19:10:00-05:00
2019-03-29 19:07:00-07:00
2019-03-29 19:10:00-07:00
2019-03-29 19:10:00-07:00
2019-03-29 19:10:00-07:00
2019-03-30 13:05:00-04:00
2019-03-30 13:05:00-04:00
2019-03-30 13:10:00-05:00
2019-03-30 14:10:00-04:00
2019-03-30 13:15:00-05:00
2019-03-30 15:07:00-04:00
2019-03-30 16:05:00-04:00
2019-03-30 18:10:00-04:00
2019-03-30 18:10:00-04:00
2019-03-30 18:10:00-05:00
2019-03-30 19:05:00-05:00
2019-03-30 17:40:00-07:00
2019-03-30 18:07:00-07:00
2019-03-30 1

2019-04-27 17:10:00-07:00
2019-04-27 18:10:00-07:00
2019-04-27 18:10:00-07:00
2019-04-28 13:05:00-04:00
2019-04-28 13:05:00-04:00
2019-04-28 13:07:00-04:00
2019-04-28 13:10:00-04:00
2019-04-28 13:20:00-04:00
2019-04-28 13:35:00-04:00
2019-04-28 13:10:00-05:00
2019-04-28 13:10:00-05:00
2019-04-28 13:15:00-05:00
2019-04-28 13:15:00-05:00
2019-04-28 13:05:00-07:00
2019-04-28 13:10:00-07:00
2019-04-28 13:10:00-07:00
2019-04-28 13:10:00-07:00
2019-04-28 18:05:00-05:00
2019-04-29 19:05:00-04:00
2019-04-29 19:10:00-04:00
2019-04-29 19:10:00-04:00
2019-04-29 19:20:00-04:00
2019-04-29 18:40:00-05:00
2019-04-29 18:40:00-05:00
2019-04-29 19:10:00-05:00
2019-04-29 19:15:00-05:00
2019-04-29 18:45:00-07:00
2019-04-30 19:05:00-04:00
2019-04-30 19:05:00-04:00
2019-04-30 19:10:00-04:00
2019-04-30 19:10:00-04:00
2019-04-30 19:10:00-04:00
2019-04-30 19:20:00-04:00
2019-04-30 18:40:00-05:00
2019-04-30 18:40:00-05:00
2019-04-30 19:05:00-05:00
2019-04-30 19:10:00-05:00
2019-04-30 19:15:00-05:00
2019-04-30 1

2019-06-01 19:10:00-07:00
2019-06-02 13:05:00-04:00
2019-06-02 13:10:00-04:00
2019-06-02 13:10:00-04:00
2019-06-02 13:20:00-04:00
2019-06-02 13:35:00-04:00
2019-06-02 13:10:00-05:00
2019-06-02 13:15:00-05:00
2019-06-02 14:05:00-05:00
2019-06-02 13:10:00-06:00
2019-06-02 13:07:00-07:00
2019-06-02 13:10:00-07:00
2019-06-02 13:10:00-07:00
2019-06-02 13:10:00-07:00
2019-06-02 15:10:00-07:00
2019-06-02 19:05:00-04:00
2019-06-03 18:40:00-07:00
2019-06-03 19:10:00-07:00
2019-06-03 19:10:00-07:00
2019-06-04 19:05:00-04:00
2019-06-04 19:05:00-04:00
2019-06-04 19:07:00-04:00
2019-06-04 19:10:00-04:00
2019-06-04 19:10:00-04:00
2019-06-04 19:10:00-04:00
2019-06-04 18:40:00-05:00
2019-06-04 19:05:00-05:00
2019-06-04 19:05:00-05:00
2019-06-04 19:15:00-05:00
2019-06-04 19:15:00-05:00
2019-06-04 18:40:00-07:00
2019-06-04 19:07:00-07:00
2019-06-04 19:10:00-07:00
2019-06-04 19:10:00-07:00
2019-06-05 13:05:00-04:00
2019-06-05 12:40:00-07:00
2019-06-05 12:40:00-07:00
2019-06-05 19:05:00-04:00
2019-06-05 1

2019-07-07 13:10:00-04:00
2019-07-07 13:10:00-04:00
2019-07-07 13:20:00-04:00
2019-07-07 13:35:00-04:00
2019-07-07 13:35:00-04:00
2019-07-07 13:10:00-05:00
2019-07-07 13:10:00-05:00
2019-07-07 13:10:00-05:00
2019-07-07 13:05:00-07:00
2019-07-07 13:10:00-07:00
2019-07-07 13:10:00-07:00
2019-07-07 13:10:00-07:00
2019-07-11 19:05:00-05:00
2019-07-12 13:20:00-05:00
2019-07-12 18:05:00-04:00
2019-07-12 19:05:00-04:00
2019-07-12 19:05:00-04:00
2019-07-12 19:10:00-04:00
2019-07-12 19:10:00-04:00
2019-07-12 19:10:00-04:00
2019-07-12 19:05:00-05:00
2019-07-12 19:10:00-05:00
2019-07-12 19:15:00-05:00
2019-07-12 19:15:00-05:00
2019-07-12 18:40:00-06:00
2019-07-12 19:07:00-07:00
2019-07-12 19:07:00-07:00
2019-07-12 19:10:00-07:00
2019-07-13 13:05:00-04:00
2019-07-13 13:20:00-05:00
2019-07-13 13:07:00-07:00
2019-07-13 18:10:00-04:00
2019-07-13 19:05:00-04:00
2019-07-13 19:10:00-04:00
2019-07-13 19:15:00-04:00
2019-07-13 18:15:00-05:00
2019-07-13 18:15:00-05:00
2019-07-13 19:15:00-04:00
2019-07-13 1

2019-08-04 13:10:00-05:00
2019-08-04 13:20:00-05:00
2019-08-04 14:05:00-05:00
2019-08-04 13:10:00-06:00
2019-08-04 13:07:00-07:00
2019-08-04 13:10:00-07:00
2019-08-04 13:10:00-07:00
2019-08-04 19:05:00-04:00
2019-08-05 16:10:00-04:00
2019-08-05 19:05:00-04:00
2019-08-05 19:05:00-04:00
2019-08-05 19:10:00-04:00
2019-08-05 19:10:00-04:00
2019-08-05 19:10:00-04:00
2019-08-05 19:10:00-04:00
2019-08-05 19:10:00-04:00
2019-08-05 19:05:00-05:00
2019-08-05 19:10:00-05:00
2019-08-05 18:40:00-07:00
2019-08-05 18:45:00-07:00
2019-08-05 19:10:00-07:00
2019-08-06 19:05:00-04:00
2019-08-06 19:05:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:10:00-04:00
2019-08-06 19:05:00-05:00
2019-08-06 19:10:00-05:00
2019-08-06 19:10:00-05:00
2019-08-06 18:40:00-07:00
2019-08-06 18:45:00-07:00
2019-08-06 19:10:00-07:00
2019-08-06 19:10:00-07:00
2019-08-07 12:10:00-04:00
2019-08-07 13:10:00-04:00
2019-08-07 1

...

2020-09-13 13:10:00-05:00
2020-09-13 13:10:00-05:00
2020-09-13 13:15:00-05:00
2020-09-13 13:35:00-05:00
2020-09-13 15:07:00-04:00
2020-09-13 13:10:00-06:00
2020-09-13 13:10:00-07:00
2020-09-13 13:10:00-07:00
2020-09-13 17:08:00-07:00
2020-09-14 16:10:00-04:00
2020-09-14 16:10:00-05:00
2020-09-14 19:35:00-04:00
2020-09-14 19:10:00-05:00
2020-09-14 18:10:00-07:00
2020-09-15 18:40:00-04:00
2020-09-15 18:40:00-04:00
2020-09-15 18:40:00-04:00
2020-09-15 19:05:00-04:00
2020-09-15 19:05:00-04:00
2020-09-15 19:10:00-04:00
2020-09-15 19:35:00-04:00
2020-09-15 18:40:00-05:00
2020-09-15 19:10:00-05:00
2020-09-15 19:10:00-05:00
2020-09-15 19:15:00-05:00
2020-09-15 18:40:00-06:00
2020-09-15 18:10:00-07:00
2020-09-15 18:40:00-07:00
2020-09-15 18:40:00-07:00
2020-09-16 13:10:00-04:00
2020-09-16 13:10:00-06:00
2020-09-16 13:10:00-07:00
2020-09-16 16:10:00-05:00
2020-09-16 18:40:00-04:00
2020-09-16 18:40:00-04:00
2020-09-16 19:05:00-04:00
2020-09-16 19:05:00-04:00
2020-09-16 19:10:00-04:00
2020-09-16 1

2021-04-12 18:45:00-05:00
2021-04-12 19:10:00-05:00
2021-04-12 19:10:00-05:00
2021-04-12 19:10:00-05:00
2021-04-12 18:40:00-07:00
2021-04-12 18:45:00-07:00
2021-04-13 13:10:00-05:00
2021-04-13 12:40:00-07:00
2021-04-13 16:05:00-04:00
2021-04-13 16:10:00-04:00
2021-04-13 18:35:00-04:00
2021-04-13 19:07:00-04:00
2021-04-13 19:10:00-04:00
2021-04-13 19:20:00-04:00
2021-04-13 18:40:00-05:00
2021-04-13 18:45:00-05:00
2021-04-13 19:10:00-05:00
2021-04-13 19:10:00-05:00
2021-04-13 19:10:00-05:00
2021-04-13 18:45:00-07:00
2021-04-13 19:10:00-07:00
2021-04-14 13:07:00-04:00
2021-04-14 12:15:00-05:00
2021-04-14 12:40:00-05:00
2021-04-14 13:10:00-05:00
2021-04-14 13:10:00-05:00
2021-04-14 12:45:00-07:00
2021-04-14 18:35:00-04:00
2021-04-14 19:05:00-04:00
2021-04-14 18:10:00-05:00
2021-04-14 19:10:00-04:00
2021-04-14 19:10:00-04:00
2021-04-14 19:20:00-04:00
2021-04-14 19:10:00-05:00
2021-04-14 19:10:00-07:00
2021-04-15 12:10:00-04:00
2021-04-15 12:20:00-04:00
2021-04-15 12:35:00-04:00
2021-04-15 1

2021-05-06 18:38:00-07:00
2021-05-07 13:20:00-05:00
2021-05-07 19:05:00-04:00
2021-05-07 19:05:00-04:00
2021-05-07 19:10:00-04:00
2021-05-07 19:10:00-04:00
2021-05-07 19:10:00-04:00
2021-05-07 19:10:00-04:00
2021-05-07 19:20:00-04:00
2021-05-07 19:05:00-05:00
2021-05-07 19:10:00-05:00
2021-05-07 19:10:00-05:00
2021-05-07 19:15:00-05:00
2021-05-07 18:38:00-07:00
2021-05-07 18:40:00-07:00
2021-05-07 18:45:00-07:00
2021-05-08 13:05:00-04:00
2021-05-08 13:15:00-05:00
2021-05-08 13:20:00-05:00
2021-05-08 13:05:00-07:00
2021-05-08 13:07:00-07:00
2021-05-08 16:10:00-04:00
2021-05-08 18:10:00-04:00
2021-05-08 18:10:00-04:00
2021-05-08 19:05:00-04:00
2021-05-08 18:05:00-05:00
2021-05-08 19:10:00-04:00
2021-05-08 18:10:00-05:00
2021-05-08 18:10:00-05:00
2021-05-08 19:20:00-04:00
2021-05-08 18:07:00-07:00
2021-05-09 13:05:00-04:00
2021-05-09 13:05:00-04:00
2021-05-09 13:10:00-04:00
2021-05-09 13:10:00-04:00
2021-05-09 13:10:00-05:00
2021-05-09 13:10:00-05:00
2021-05-09 13:15:00-05:00
2021-05-09 1

2021-05-30 13:10:00-07:00
2021-05-30 13:10:00-07:00
2021-05-30 13:10:00-07:00
2021-05-31 13:05:00-04:00
2021-05-31 13:05:00-04:00
2021-05-31 13:10:00-05:00
2021-05-31 14:10:00-04:00
2021-05-31 13:20:00-05:00
2021-05-31 15:10:00-04:00
2021-05-31 13:05:00-07:00
2021-05-31 13:10:00-07:00
2021-05-31 15:10:00-05:00
2021-05-31 17:10:00-04:00
2021-05-31 19:10:00-05:00
2021-05-31 18:10:00-07:00
2021-05-31 18:40:00-07:00
2021-06-01 18:10:00-04:00
2021-06-01 19:05:00-04:00
2021-06-01 19:05:00-04:00
2021-06-01 19:07:00-04:00
2021-06-01 19:10:00-04:00
2021-06-01 19:20:00-04:00
2021-06-01 18:40:00-05:00
2021-06-01 19:05:00-05:00
2021-06-01 19:10:00-05:00
2021-06-01 19:10:00-05:00
2021-06-01 18:40:00-06:00
2021-06-01 18:40:00-07:00
2021-06-01 18:45:00-07:00
2021-06-01 19:10:00-07:00
2021-06-01 19:10:00-07:00
2021-06-02 12:35:00-04:00
2021-06-02 13:20:00-05:00
2021-06-02 12:40:00-07:00
2021-06-02 19:05:00-04:00
2021-06-02 19:05:00-04:00
2021-06-02 19:07:00-04:00
2021-06-02 19:20:00-04:00
2021-06-02 1

2021-06-25 19:10:00-04:00
2021-06-25 19:10:00-04:00
2021-06-25 19:10:00-04:00
2021-06-25 19:05:00-05:00
2021-06-25 19:10:00-05:00
2021-06-25 19:10:00-05:00
2021-06-25 19:15:00-05:00
2021-06-25 18:45:00-07:00
2021-06-25 19:10:00-07:00
2021-06-25 19:10:00-07:00
2021-06-26 13:10:00-05:00
2021-06-26 13:15:00-05:00
2021-06-26 15:07:00-04:00
2021-06-26 15:05:00-05:00
2021-06-26 16:10:00-04:00
2021-06-26 15:10:00-05:00
2021-06-26 16:10:00-04:00
2021-06-26 16:10:00-04:00
2021-06-26 16:10:00-04:00
2021-06-26 18:10:00-04:00
2021-06-26 16:15:00-07:00
2021-06-26 19:15:00-04:00
2021-06-26 19:05:00-07:00
2021-06-26 19:10:00-07:00
2021-06-27 13:07:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-04:00
2021-06-27 13:10:00-05:00
2021-06-27 13:10:00-05:00
2021-06-27 13:15:00-05:00
2021-06-27 13:35:00-05:00
2021-06-27 13:05:00-07:00
2021-06-27 13:10:00-07:00
2021-06-27 16:15:00-05:00
2021-06-27 1

In [13]:
game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))

In [ ]:
game_pk_df

In [14]:
datetime_coordinate_matching = {}
for index, row in game_pk_df.iterrows():
    city, lat, lon, alt = venue_dict[row['venue_name']]
    dc_datetime = row['local_datetime']
    if dc_datetime in datetime_coordinate_matching.keys():
        datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
    else:
        datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]
#     print(index/4524)

In [ ]:
game_pk_df['venue_name']

In [15]:
game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))

In [ ]:
for key, value in datetime_coordinate_matching.items():
    print(value)

In [ ]:
from api_keys import visual_crossing
import requests

weather_df = pd.DataFrame()

for key, value in datetime_coordinate_matching.items():
    url_locations = '|'.join(value)
    URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&location={url_locations}&key={visual_crossing}'
    try:
        response = requests.get(URL)
        data = response.json()
        locations = list(data['locations'].keys())
        for each in locations:
            values = data['locations'][each]['values'][0]
            weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
    except:
        raise IndexError(key, value, response)
#     [list(response.json()['locations'].keys())[0]]['values'][0]
#     break
#     data = data.decode('utf-8').splitlines()
#     if data[0][:7] != 'Address':
#         print(data)
#         break
#     data = dict(zip(str.split(data[0], ','), str.split(data[1], ',')))
#     weather_df = weather_df.append(data, ignore_index=True)
    
    
# return(url_datetime, data)

# game_pk_df.apply(lambda row: vc_weather(row))



In [ ]:
weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
weather_df

Important to save since this actually cost money

In [ ]:
weather_df.to_csv('weather_df')

In [16]:
weather_df = pd.read_csv('weather_df')
# df_filtered = pd.read_csv('df_filtered')

In [ ]:
df_filtered

In [ ]:
game_pk_df

In [17]:
games_and_weather = pd.merge(
    game_pk_df,
    weather_df,
    how="outer",
    on=None,
    left_on=['local_datetime', 'coordinates'],
    right_on=['datetimeStr', 'coordinates'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_acw"),
    copy=True,
    indicator=False,
    validate=None,
)

In [ ]:
games_and_weather

In [18]:
df_detailed = pd.merge(
    games_and_weather,
    df_filtered,
    how="outer",
    on=None,
    left_on='game_id',
    right_on='game_pk',
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_acw"),
    copy=True,
    indicator=False,
    validate=None,
)

In [ ]:
df_detailed.info()

In [ ]:
df_detailed.describe()

In [ ]:
df_detailed[df_detailed['temp'] > 114]['venue_name'].value_counts()

In [ ]:
df_detailed[df_detailed['at_bat_number'] == 148 ]

In [ ]:
df_detailed[df_detailed['delta_home_win_exp'] > .9]

In [19]:
df_detailed['covid_doubleheader'] = df_detailed.apply(lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] =='Y' else 0, axis=1)

In [20]:
american_league_teams = ['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles']

In [21]:
df_detailed['designated_hitter'] = df_detailed.apply(lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis = 1)

In [ ]:
df_detailed[(df_detailed['home_name'] == 'New York Mets') & (df_detailed['designated_hitter'] == 1)]

In [ ]:
df_detailed[(df_detailed['home_name'] == 'Oakland Athletics') & (df_detailed['designated_hitter'] == 0)]

next step: create a function which can take in a player (name or player_id?), type of stats requested (pitcher or hitter), a start date, and an end date, and can return cumulated stats through that time period

In [23]:
# probably best off grabbing all the data for each player and trimming it myself - speed up, cut down on api calls

# instead of career stats we'll go off last two years to bias towards recency
def derived_cumulative_stats(player, stats_type, database, start_date=None, end_date='2021-06-30'):
    print(end_date)
    if start_date == None:
        start_date = int(end_date[:4]) - 2
        start_date = str(start_date) + end_date[4:]
#     start_date = start_date.strftime('%Y-%m-%d')
#     end_date = end_date.strftime('%Y-%m-%d')
#     if type(player) == str:
#         player = player.split(', ')
#         try:
#             player = playerid_lookup(player[0], player[1])
#             if len(player) >1:
#                 raise NameError(player)
#             else:
#                 player = player['key_mlbam'].iloc[0]
#         except:
#             raise NameError(player)
    history = database
    
#     alternative datasource - statcast api
#     return(statcast_pitcher(start_date, end_date, player))
#     history = statcast_pitcher(start_date, end_date, player)
#     return(history)
#     history = history.sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number']).reset_index(drop=True)


    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]
#     return(history)
    if stats_type == 'pitcher':
        history = history[history['pitcher'] == player].reset_index(drop=True)
        history = history[history['game_type'] == 'R']
    #     derive hits per inning, batters faced per appearence,
    #     derive R/L splits for all the above
    #     pitch specific stats - assemble pitch types in df with how often thrown, avg pfx_x, pfx_z, vx0, vy0, vz0, ax, ay, az, release_spin_rate, release_extension

    #   PITCH METRICS
        pitch_list = [x for x in history['pitch_type'].value_counts().index]
        pitches = {}
        for each in pitch_list:
            if type(each) == float:
                each = 'UNKNOWN'
            metrics = history[history['pitch_type'] == each].describe()
            pitches[each+'_quantity'] =  metrics.loc['count', 'batter']
            pitches[each+'_avg_release_speed'] = metrics.loc['mean', 'release_speed']
            pitches[each+'_avg_pfx_x'] = metrics.loc['mean', 'pfx_x']
            pitches[each+'_avg_pfx_z'] = metrics.loc['mean', 'pfx_z']
            pitches[each+'_avg_vx0'] = metrics.loc['mean', 'vx0']
            pitches[each+'_avg_vy0'] = metrics.loc['mean', 'vy0']
            pitches[each+'_avg_vz0'] = metrics.loc['mean', 'vz0']
            pitches[each+'_avg_ax'] = metrics.loc['mean', 'ax']
            pitches[each+'_avg_ay'] = metrics.loc['mean', 'ay']
            pitches[each+'_avg_az'] = metrics.loc['mean', 'az']
            pitches[each+'_avg_release_spin_rate'] = metrics.loc['mean', 'release_spin_rate']
            pitches[each+'_avg_release_extension'] = metrics.loc['mean', 'release_extension']

    #     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]

    #    PLAYER METRICS
        games = history['game_pk'].unique()
        at_bat_list = []
#         inning_list = []
        num_hits_list = []
        for game in games:
            inning = history[history['game_pk'] == game].tail(1)['inning'].iloc[0]
#             inning_list.append(inning)
            for i in range(1, inning+1):
                num_hits = len(history[(history['game_pk'] == game) & (history['inning'] == i) & (history['events'].isin(hit_events))])
                num_hits_list.append(num_hits)
            at_bats = len(history[history['game_pk'] == game])
            at_bat_list.append(at_bats)
        pitches['avg_PAs_per_apparence'] = np.array(at_bat_list).mean()
        pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()
        
#         L/R splits
        l_pas = len(history[history['stand'] == 'L'])
        r_pas = len(history[history['stand'] == 'R'])
        l_hits = len(history[(history['stand'] == 'L') & (history['events'].isin(hit_events))])
        r_hits = len(history[(history['stand'] == 'R') & (history['events'].isin(hit_events))])
        
        if (l_pas + r_pas) > 0:
            pitches['H/PA'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        else:
            pitches['H/PA'] = 0
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = 0
        return(pitches)
    
    elif stats_type == 'batter':
        history = history[history['pitcher'] == player].reset_index(drop=True)

        history = history[history['game_type'] == 'R']

#     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]
    
#     H/PA per pitch type
#     K/PA per pitch type
#     H/PA vs righties, lefties
#     PA/G
#     avg_launch_angle
#     avg_launch_speed
#     xBA based on estimated_ba_using_speedangle



        pitches = history['pitch_type'].unique()
        games = history['game_pk'].unique()
        pas_list = []
        
        batter = {}
        
        for game in games:
            pas = len(history['game_pk'] == game)
            pas_list.append(pas)
        
        batter['PA/G'] = np.array(pas_list).mean()
        
        for pitch in pitches:
            if type(pitch) == float:
                pitch = 'UNKNOWN'
            pas = len(history['pitch_type'] == pitch)
            hits = len(history[(history['pitch_type'] == pitch) & history['events'].isin(hit_events)])
            strikeouts = len(history[(history['pitch_type'] == pitch)& (history['events'].isin(['strikeout', 'strikeout_double_play']))])
            
            if pas > 0:
                batter[pitch+'_H/PA'] = hits/pas
                batter[pitch+'_K/PA'] = strikeouts/pas
            else:
                batter[pitch+'_H/PA'] = 0
                batter[pitch+'_K/PA'] = 0
#         righty/lefty split
        pas_r = len(history[history['p_throws'] == 'R'])
        hits_r = len(history[(history['p_throws'] == 'R') & history['events'].isin(hit_events)])
        
        if pas_r > 0:
            batter['H/PA_against_R'] = hits_r/pas_r
        else:
            batter['H/PA_against_R'] = 0
        
        pas_l = len(history[history['p_throws'] == 'L'])
        hits_l = len(history[(history['p_throws'] == 'L') & history['events'].isin(hit_events)])
        
        if pas_l > 0:
            batter['H/PA_against_L'] = hits_l/pas_l
        else:
            batter['H/PA_against_L'] = 0
        
        batter['avg_launch_angle'] = history['launch_angle'].mean()
        batter['avg_launch_speed'] = history['launch_speed'].mean()
        batter['xBA'] = history['estimated_ba_using_speedangle'].mean()
        
        return(batter)

# senario_df = senario_df.sort_values('hit').drop_duplicates(subset=['date', 'number', 'player'], keep='last')


In [10]:
# probably best off grabbing all the data for each player and trimming it myself - speed up, cut down on api calls

# instead of career stats we'll go off last two years to bias towards recency
def derived_cumulative_stats_v2(player, stats_type, history, start_date=None, end_date='2021-06-30'):
    print(end_date)
    if start_date == None:
        start_date = int(end_date[:4]) - 2
        start_date = str(start_date) + end_date[4:]
        
    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]
    if stats_type == 'pitcher':
        history = history[history['pitcher'] == player]
        history = history[history['game_type'] == 'R']
    #     derive hits per inning, batters faced per appearence,
    #     derive R/L splits for all the above
    #     pitch specific stats - assemble pitch types in df with how often thrown, avg pfx_x, pfx_z, vx0, vy0, vz0, ax, ay, az, release_spin_rate, release_extension

    #   PITCH METRICS
        pitch_list = history['pitch_type'].unique()
        pitches = {}
        for each in pitch_list:
            if type(each) == float:
                each = 'UNKNOWN'
            metrics = history[history['pitch_type'] == each].describe()
            pitches[each+'_quantity'] =  metrics.loc['count', 'batter']
            pitches[each+'_avg_release_speed'] = metrics.loc['mean', 'release_speed']
            pitches[each+'_avg_pfx_x'] = metrics.loc['mean', 'pfx_x']
            pitches[each+'_avg_pfx_z'] = metrics.loc['mean', 'pfx_z']
            pitches[each+'_avg_vx0'] = metrics.loc['mean', 'vx0']
            pitches[each+'_avg_vy0'] = metrics.loc['mean', 'vy0']
            pitches[each+'_avg_vz0'] = metrics.loc['mean', 'vz0']
            pitches[each+'_avg_ax'] = metrics.loc['mean', 'ax']
            pitches[each+'_avg_ay'] = metrics.loc['mean', 'ay']
            pitches[each+'_avg_az'] = metrics.loc['mean', 'az']
            pitches[each+'_avg_release_spin_rate'] = metrics.loc['mean', 'release_spin_rate']
            pitches[each+'_avg_release_extension'] = metrics.loc['mean', 'release_extension']

    #     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]

    #    PLAYER METRICS
        games = history['game_pk'].unique()
        at_bat_list = []
        num_hits_list = []
        for game in games:
            game = history[history['game_pk'] == game]
            inning = game['inning'].max()
            at_bats = len(game)
            at_bat_list.append(at_bats)
            for i in range(1, inning+1):
                num_hits = len(game[(history['inning'] == i) & (history['events'].isin(hit_events))])
                num_hits_list.append(num_hits)
        pitches['avg_PAs_per_apparence'] = np.array(at_bat_list).mean()
        pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()
        
#         L/R splits
        left_pitcher = history[history['stand'] == 'L']
        right_pitcher = history[history['stand'] == 'R']
        l_pas = len(left_pitcher)
        r_pas = len(right_pitcher)
        l_hits = len(left_pitcher[(history['events'].isin(hit_events))])
        r_hits = len(right_pitcher[(history['events'].isin(hit_events))])
        
        if (l_pas + r_pas) > 0:
            pitches['H/PA'] = (l_hits + r_hits)/(l_pas + r_pas)
            pitches['against_lefties_H/PA'] = (l_hits)/(l_pas)
            pitches['against_righties_H/PA'] = (r_hits)/(r_pas)
        else:
            pitches['H/PA'] = 0
            pitches['against_lefties_H/PA'] = 0
            pitches['against_righties_H/PA'] = 0
        return(pitches)
    
    elif stats_type == 'batter':
        history = history[history['pitcher'] == player]

        history = history[history['game_type'] == 'R']

#     filter history down to PA-enders
        history = history[history['events'].isin(pa_ending_events)]
    
#     H/PA per pitch type
#     K/PA per pitch type
#     H/PA vs righties, lefties
#     PA/G
#     avg_launch_angle
#     avg_launch_speed
#     xBA based on estimated_ba_using_speedangle



        pitches = history['pitch_type'].unique()
        games = history['game_pk'].unique()
        pas_list = []
        
        batter = {}
        
        for game in games:
            pas = len(history['game_pk'] == game)
            pas_list.append(pas)
        
        batter['PA/G'] = np.array(pas_list).mean()
        
        for pitch in pitches:
            if type(pitch) == float:
                pitch = 'UNKNOWN'
            pt = history['pitch_type'] == pitch
            pas = len(pt)
            hits = len(pt[history['events'].isin(hit_events)])
            strikeouts = len(pt[(history['events'].isin(['strikeout', 'strikeout_double_play']))])
            
            if pas > 0:
                batter[pitch+'_H/PA'] = hits/pas
                batter[pitch+'_K/PA'] = strikeouts/pas
            else:
                batter[pitch+'_H/PA'] = 0
                batter[pitch+'_K/PA'] = 0
#         righty/lefty split
        pitcher_right = history[history['p_throws'] == 'R']
        pas_r = len(pitcher_right)
        hits_r = len(pitcher_right[history['events'].isin(hit_events)])
        
        if pas_r > 0:
            batter['H/PA_against_R'] = hits_r/pas_r
        else:
            batter['H/PA_against_R'] = 0
        
        pitcher_left = history[history['p_throws'] == 'L']
        pas_l = len(pitcher_left)
        hits_l = len(pitcher_left[history['events'].isin(hit_events)])
        
        if pas_l > 0:
            batter['H/PA_against_L'] = hits_l/pas_l
        else:
            batter['H/PA_against_L'] = 0
        
        batter['avg_launch_angle'] = history['launch_angle'].mean()
        batter['avg_launch_speed'] = history['launch_speed'].mean()
        batter['xBA'] = history['estimated_ba_using_speedangle'].mean()
        
        return(batter)

# senario_df = senario_df.sort_values('hit').drop_duplicates(subset=['date', 'number', 'player'], keep='last')


In [ ]:
# playerid_lookup('deGrom, Jacob'.split(', '))

# type('degrom, jacob') == str

# database = pd.concat([pd.read_csv('2017_statcast_pbp.csv'),
#                          pd.read_csv('2018_statcast_pbp.csv'),
#                 pd.read_csv('2019_statcast_pbp.csv'),
#                 pd.read_csv('2020_statcast_pbp.csv'),
#                pd.read_csv('2021_statcast_pbp.csv')], ignore_index=True)

derived_cumulative_stats('degrom, jacob', 'pitcher', database)

In [ ]:
'2019-05-19' > '2019-04-19'

In [6]:
database = pd.concat([pd.read_csv('2017_statcast_pbp.csv'),
                         pd.read_csv('2018_statcast_pbp.csv'),
                pd.read_csv('2019_statcast_pbp.csv'),
                pd.read_csv('2020_statcast_pbp.csv'),
               pd.read_csv('2021_statcast_pbp.csv')], ignore_index=True).sort_values(['game_date', 'game_pk','home_team', 'inning', 'outs_when_up', 'at_bat_number', 'pitch_number'])

In [14]:
# %%timeit -n 3
derived_detailed_stats_2yr = df_detailed.iloc[0:1].apply(lambda row: pd.Series({**derived_cumulative_stats_v2(row['pitcher'], 'pitcher', database, end_date=datetime.strftime(datetime.strptime(row['game_date_gpk'], '%Y-%m-%d')-timedelta(days=1), '%Y-%m-%d')), **derived_cumulative_stats_v2(row['batter'], 'batter', database, end_date=datetime.strftime(datetime.strptime(row['game_date_gpk'], '%Y-%m-%d')-timedelta(days=1), '%Y-%m-%d'))}), axis = 1)

import warnings
warnings.filterwarnings('ignore')


2019-03-19
2019-03-19


In [22]:
pd.DataFrame(venue_dict).to_csv('venue_dict')
df_detailed.to_csv('df_detailed')
game_pk_df.to_csv('game_pk_df')

In [3]:
df_detailed = pd.read_csv('df_detailed')
game_pk_df = pd.read_csv('game_pk_df')

/Users/home/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (2,3,4,5,6,7,10,12,13,14,15,19,21,22,23,24,25,26,27,28,32,34,51,55,56,60,63,64,70,71,72,73,74,75,76,78,132,141,142) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
database[database['game_pk'] == 633506]['inning'].max()

7

In [32]:
len(df_detailed)

345467